In [ ]:
#@title **<- Install dependencies and Mount Gdrive** { vertical-output: true, display-mode: "form" }
!apt install python3-libtorrent
!pip3 install lbry-libtorrent

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title **<- Click the play button for start download** { vertical-output: true, display-mode: "form" }
#@markdown <br><center><h1><b>TTGD</b></h1></ceneter>
#@markdown <center><img src='https://www.libtorrent.org/img/logo-color-text.png' height="50" alt="libtorrent-logo"/><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/c/c3/Python-logo-notext.svg/121px-Python-logo-notext.svg.png' height="50" alt="python-logo"/><img src='https://ssl.gstatic.com/images/branding/product/1x/drive_2020q4_48dp.png' height="50" alt="Drive-logo"/></center><br>

magnetLink = "" #@param {type:"string"}
gdriveDownloadPath = "TTGD" #@param {type:"string"}
compressionMethod = "TAR" #@param ["TAR", "TAR.XZ"]

#@markdown <br><h1><b>Read Me<b></h1>
#@markdown - TAR compression method
#@markdown  - Lower compression ratio
#@markdown  - Lower compression time
#@markdown - TAR.XZ compression method
#@markdown  - Higher compression ratio
#@markdown  - Higher compression time<br>

import libtorrent as lt
import shutil, string, time, re,  os

gdriveDownloadPath = os.path.join("/content/drive/MyDrive", gdriveDownloadPath)
torrentDownloadPath = "/content/Work_space/DTORRENT/"
torrentCompressPath = "/content/Work_space/CTORRENT/"
chars = re.escape(string.punctuation)

if not os.path.isdir(gdriveDownloadPath):
  os.mkdir(gdriveDownloadPath)

if compressionMethod == "TAR":
  cMethod = "tar"
  cExtention = ".tar"
elif compressionMethod == "TAR.XZ":
  cMethod = "xztar"
  cExtention = ".tar.xz"

downloadConfig = {
    'save_path': torrentDownloadPath,
    'storage_mode': lt.storage_mode_t(2)}

SESSION = lt.session()
SESSION.listen_on(6881, 6891)
DOWNLOADER = lt.add_magnet_uri(SESSION, magnetLink, downloadConfig)
SESSION.start_dht()

print("\n[*]Collecting metadata...")

while not DOWNLOADER.has_metadata():
  time.sleep(1)

NAME = str(DOWNLOADER.status().name)

print("\nMetadata collected...")
print(f"[*]{NAME} downloading started...\n")

while not DOWNLOADER.status().is_seeding:
  update = DOWNLOADER.status()

  print(f"[!]{round(update.progress*100, 2):5n}% completed | download_rate: {round(update.download_rate/1000000, 2):5n} MB/s | upload_rate: {round(update.upload_rate/1000000, 2):5n} MB/s | peers_connected: {(update.num_peers):3n} | state: {update.state}")

  time.sleep(5)

print("\n[*]Download completed...")
print(f"\n[*]Adding files to a {compressionMethod} file...")

encNAME = str(re.sub(r'['+chars+']', '',(NAME.encode("ascii", "ignore")).decode())).replace(" ", "_")

shutil.move(f"{torrentDownloadPath}{NAME}", f"{torrentDownloadPath}{encNAME}")

shutil.make_archive(f"{torrentCompressPath}{encNAME}", cMethod, f"{torrentDownloadPath}{encNAME}")

print("\n[*]Moving compressed files to Gdrive...")

shutil.move(f"{torrentCompressPath}{encNAME}{cExtention}", f"{gdriveDownloadPath}/{encNAME}{cExtention}")

print("\n[*]Cleaning the disk...")

shutil.rmtree(f"{torrentDownloadPath}{encNAME}", ignore_errors=True)

print("\n[*]Finished.")
print("\n[!]Before you leave please make sure to Disconnect and delete the runtime")

# **Find me on**<br>
- Youtube : [@mrTmash](https://www.youtube.com/c/mrTmash00)
- Github : [@mrtmash](https://github.com/mrtmash)

### <br>**Before you leave please make sure to Disconnect and delete the runtime**